In [20]:
import pandas as pd
import numpy as np

In [21]:
df = pd.read_csv('dataset_games.csv')
t = df
del df['release_date']
del df['Unnamed: 0']

In [22]:
df['description'] = df['description'].fillna('None')
df.head()

,name_game,meta_score,user_score,platform,description
0,The Legend of Zelda: Ocarina of Time,99,9.1,Nintendo 64,"As a young boy, Link is tricked by Ganondorf, ..."
1,Tony Hawk's Pro Skater 2,98,7.5,PlayStation,As most major publishers' development efforts ...
2,Grand Theft Auto IV,98,7.7,PlayStation 3,[Metacritic's 2008 PS3 Game of the Year; Also ...
3,SoulCalibur,98,8.5,Dreamcast,"This is a tale of souls and swords, transcendi..."
4,Grand Theft Auto IV,98,7.9,Xbox 360,[Metacritic's 2008 Xbox 360 Game of the Year; ...


In [23]:
df['user_score'] = np.where(df['user_score'] == 'tbd',df['meta_score']/10 ,df['user_score'])
df['user_score'] = df['user_score'].astype(float)
df.dtypes

name_game       object
meta_score       int64
user_score     float64
platform        object
description     object
dtype: object

In [24]:
w = df[df.duplicated(subset='name_game', keep='first')]

In [25]:
duplicates = list(sorted(set(w['name_game'])))

In [26]:
# test

names = []
meta_score = []
user_score = []
platform = []
description = []
index = 0
ind = []

for r in zip(*t.to_dict("list").values()):
    if r[0] in duplicates and r[0] not in names:
        names.append(r[0])
        meta_score.append([r[1]])
        user_score.append([r[2]])
        platform.append([r[3]])
        description.append([r[4]])
    elif r[0] in names:
        tmp = names.index(r[0])
        meta_score[tmp].append(r[1])
        user_score[tmp].append(r[2])
        platform[tmp].append(r[3])
        description[tmp].append(r[4])

In [27]:
def checker(des):
    if des[0] == '[':
        for x in range(len(des)):
            if des[x] == ']':
                return x + 1

In [28]:

def info(name, meta_score, user_score, platform, description, index):
    avg_meta = round((sum(meta_score[index]))/len(meta_score[index]), 2)
    avg_user = round((sum(user_score[index]))/len(user_score[index]), 2)

    
    des = max(description[index], key=len)
    
    test = checker(des)
    if test != None:
        des = des[test:].lstrip()
    else:
        des = des.lstrip()
        
    return name[index], avg_meta, avg_user, platform[index], des

In [51]:
# numpy loop
name_game_ = np.array([])
meta_score_ = np.array([])
user_score_ = np.array([])
platform_ = []
description_ = np.array([])
counter = 0

for x in range(len(duplicates)):
    counter+=1
    data = info(duplicates, meta_score, user_score, platform, description, x)
    name_game_  = np.append(name_game_, data[0])
    meta_score_ = np.append(meta_score_, data[1])
    user_score_ = np.append(user_score_, data[2])
    platform_.append(data[3])
    description_ = np.append(description_, data[4])

df2 = pd.DataFrame({
    'name_game': name_game_,
    'meta_score': meta_score_,
    'user_score': user_score_,
    'platform': platform_,
    'description': description_,
})

In [53]:
df = df[~df['name_game'].isin(duplicates)].reset_index(drop=True)
df

,name_game,meta_score,user_score,platform,description
0,The Legend of Zelda: Ocarina of Time,99.0,91.0,Nintendo 64,"As a young boy, Link is tricked by Ganondorf, ..."
1,Super Mario Galaxy,97.0,91.0,Wii,[Metacritic's 2007 Wii Game of the Year] The u...
2,Super Mario Galaxy 2,97.0,91.0,Wii,"Super Mario Galaxy 2, the sequel to the galaxy..."
3,Metroid Prime,97.0,89.0,GameCube,Samus returns in a new mission to unravel the ...
4,Super Mario Odyssey,97.0,89.0,Switch,New Evolution of Mario Sandbox-Style Gameplay....
...,...,...,...,...,...
4254,SPOGS Racing,18.0,37.0,Wii,[WiiWare] Race along in distinctively unique ...
4255,Double Dragon II: Wander of the Dragons,17.0,15.0,Xbox 360,A remake the 80's Double Dragon II game. Team ...
4256,Vroom in the Night Sky,17.0,32.0,Switch,Vroom in the night sky is a magical bike actio...
4257,Yaris,17.0,44.0,Xbox 360,[Xbox Live Arcade] Hop into a Toyota Yaris an...


In [54]:
df = df.append(df2).reset_index(inplace=False, drop=True)
df

,name_game,meta_score,user_score,platform,description
0,The Legend of Zelda: Ocarina of Time,99.00,91.00,Nintendo 64,"As a young boy, Link is tricked by Ganondorf, ..."
1,Super Mario Galaxy,97.00,91.00,Wii,[Metacritic's 2007 Wii Game of the Year] The u...
2,Super Mario Galaxy 2,97.00,91.00,Wii,"Super Mario Galaxy 2, the sequel to the galaxy..."
3,Metroid Prime,97.00,89.00,GameCube,Samus returns in a new mission to unravel the ...
4,Super Mario Odyssey,97.00,89.00,Switch,New Evolution of Mario Sandbox-Style Gameplay....
...,...,...,...,...,...
8826,kill.switch,33.00,7.40,"[Xbox, Xbox]",Your favorite characters from the movie Shrek ...
8827,nail'd,30.33,3.33,"[PlayStation 3, PlayStation 3, Xbox 360]","Due for release on PSN in June, AMY is a survi..."
8828,oOo: Ascension,20.67,2.13,"[Xbox 360, PC, PlayStation 3]",The Leisure Suit Larry: Box Office Bust video ...
8829,uDraw Studio,24.50,3.05,"[Xbox 360, PlayStation 3]",Paying homage to the old school look and feel ...


In [55]:
df['user_score'] = df['user_score'].apply(lambda x: round(x * 10,2))
df.to_csv('final_dataset.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'final_dataset.csv'

In [56]:
df.loc[[df['user_score'].idxmax()]]

,name_game,meta_score,user_score,platform,description
3214,Metal Torrent,62.0,970.0,DS,[DSiWare] Prepare for a high level of intensi...


In [57]:
df['platform']

0                                    Nintendo 64
1                                            Wii
2                                            Wii
3                                       GameCube
4                                         Switch
                          ...                   
8826                                [Xbox, Xbox]
8827    [PlayStation 3, PlayStation 3, Xbox 360]
8828               [Xbox 360, PC, PlayStation 3]
8829                   [Xbox 360, PlayStation 3]
8830                              [Xbox 360, PC]
Name: platform, Length: 8831, dtype: object